In [20]:
# for evt
# for jet
# check if there exists a reconstr jet delR = sqrt(del eta^2 + del phi^2) < 0.4
# use boolean in a for loop and check if there are reconstr jets which match -> if there are -> break

In [1]:
import uproot
filename="user.tapark.41935082._000001.tree.root"
file = uproot.open(filename)
tree = file["JetConstituentTree"]
# tree.show()

# to read TBranch ; NOTE: use entry_start, entry_stop to read only specific amounts of entry for optimization
# tree.arrays(['truthJet_pt', 'truthJet_eta', 'truthJet_phi', 'truthJet_m'], library="np")
truth_jets = tree.arrays(['truthJet_pt', 'truthJet_eta', 'truthJet_phi', 'truthJet_m'], library="np", entry_stop=1000)
# add filtering on truth jets: cut="truthJet_m >= 20000"
reco_jets = tree.arrays(['jet_pt', 'jet_eta', 'jet_phi', 'jet_m'], library="np", entry_stop=1000)

# truth_jets


In [2]:
import vector
# define MomentumObject4D
def constructFourMomentum(pt_val : float | None, phi_val : float | None, eta_val : float | None, m_val: float | None) -> vector.MomentumObject4D:
    return vector.MomentumObject4D(pt=pt_val, phi=phi_val, eta=eta_val, m=m_val)

def truthVar(varType, eventNum, jetNum):
    if varType == "truthJet_pt":
        if len(truth_jets[eventNum]['truthJet_pt']) != 0:
            return truth_jets[eventNum]['truthJet_pt'][jetNum]
        else:
            return None
    if varType == "truthJet_phi":
        if len(truth_jets[eventNum]['truthJet_phi']) != 0:
            return truth_jets[eventNum]['truthJet_phi'][jetNum]
        else:
            return None
    if varType == "truthJet_eta":
        if len(truth_jets[eventNum]['truthJet_eta']) != 0:
            return truth_jets[eventNum]['truthJet_eta'][jetNum]
        else:
            return None
    if varType == "truthJet_m":
        if len(truth_jets[eventNum]['truthJet_m']) != 0:
            return truth_jets[eventNum]['truthJet_m'][jetNum]
        else:
            return None

def recoVar(varType, eventNum, jetNum):
    if varType == "jet_pt":
        if len(reco_jets[eventNum]['jet_pt']) != 0:
            return reco_jets[eventNum]['jet_pt'][jetNum]
        else:
            return None
    if varType == "jet_phi":
        if len(reco_jets[eventNum]['jet_phi']) != 0:
            return reco_jets[eventNum]['jet_phi'][jetNum]
        else:
            return None
    if varType == "jet_eta":
        if len(reco_jets[eventNum]['jet_eta']) != 0:
            return reco_jets[eventNum]['jet_eta'][jetNum]
        else:
            return None
    if varType == "jet_m":
        if len(reco_jets[eventNum]['jet_m']) != 0:
            return reco_jets[eventNum]['jet_m'][jetNum]
        else:
            return None
    

In [82]:
# import awkward as ak

# builder = ak.ArrayBuilder()

# # Append some values
# builder.append(1)  # integer
# builder.append(True)  # boolean
# builder.append(None)  # null
# builder.append({"x": 1, "y": [2, 3]})  # dictionary with list

# builder.begin_list()
# builder.append(3)
# builder.append([1,2])
# builder.end_list()

# builder.to_list()

list

In [77]:
# import awkward as ak
# comparedjetsfilename = "user.tapark.41935082._000001.tree.compareTrueRecoJets.root"
# output = uproot.update(comparedjetsfilename)

# output.extend("JetConstituentTree", {"TruthJetXIsReco": }) 

# # comparedJetsFile = uproot.open(comparedjetsfilename)
# # comparedJetsTree = comparedJetsFile["JetConstituentTree"]
# # comparedJetsTree.keys()

In [40]:
import awkward as ak
comparedjetsfilename = "user.tapark.41935082._000001.tree.compareTrueRecoJets.root"


builder = ak.ArrayBuilder()
eventShow = []

eventsNum = len(truth_jets)
# loop through events
for eventNum in range(eventsNum):
    # start a list for an event
    builder.begin_list()
    if ( len(truth_jets[eventNum]['truthJet_pt']) == 0 and
        len(truth_jets[eventNum]['truthJet_phi']) == 0 and
        len(truth_jets[eventNum]['truthJet_eta']) == 0 and
        len(truth_jets[eventNum]['truthJet_m']) == 0 ):
        continue
    else:
        l = len(truth_jets[eventNum]['truthJet_pt'])
        k = len(reco_jets[eventNum]['jet_pt'])
        for truthJetNum in range(l):
            # start a list for a truth jet
            builder.begin_list()
            var_pt = truthVar("truthJet_pt", eventNum, truthJetNum)
            var_phi = truthVar("truthJet_phi", eventNum, truthJetNum)
            var_eta = truthVar("truthJet_eta", eventNum, truthJetNum)
            var_m = truthVar("truthJet_m", eventNum, truthJetNum)
            m20GeV_flag = False
            if var_m >= 20000:
                m20GeV_flag = True
            p_truth = constructFourMomentum(var_pt, var_phi, var_eta, var_m)
            
            for recoJetNum in range(k):
                var_pt = recoVar("jet_pt", eventNum, recoJetNum)
                var_phi = recoVar("jet_phi", eventNum, recoJetNum)
                var_eta = recoVar("jet_eta", eventNum, recoJetNum)
                var_m = recoVar("jet_m", eventNum, recoJetNum)
                p_reco = constructFourMomentum(var_pt, var_phi, var_eta, var_m)
                dR = p_truth.deltaR(p_reco)
                if dR < 0.4 and m20GeV_flag:
                    builder.append(1)
                    eventShow.append(1)
                elif dR < 0.4:
                    builder.append(-1)
                else:
                    builder.append(0)
            # close truth jet list
            builder.end_list()
    # close event list
    builder.end_list()

# eventList = builder.to_list()
eventShow
# output = uproot.recreate(comparedjetsfilename)
# output.mktree("TruthJetXIsReco", {"IsReco": builder.to_list.type})
# output["TruthJetXIsReco"].exnted({"IsReco": builder.to_list})

: 

In [ ]:
# TO DO: Histogram over eta of all truth jets vs matched jets